#Predicting sentiment from product reviews

#Fire up GraphLab Create

In [1]:
import graphlab

#Read some product review data

Loading reviews for a set of baby products. 

In [2]:
products = graphlab.SFrame('amazon_baby.gl/')

[INFO] This non-commercial license of GraphLab Create is assigned to iliassweb@gmail.comand will expire on September 22, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-25262 - Server binary: /home/zax/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1445838606.log
[INFO] GraphLab Server Version: 1.6.1


#Let's explore this data together

Data includes the product name, the review text and the rating of the review. 

In [3]:
products.head()

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


#Build the word count vector for each review

In [4]:
products['word_count'] = graphlab.text_analytics.count_words(products['review'])


In [6]:
products.head()

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'and': 5, 'stink': 1,'because': 1, 'ordered': ..."
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3, 'love': 1,'it': 2, 'highly': 1, ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2, 'quilt': 1,'it': 1, 'comfortable': ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1, 'and':3, 'love': 2, ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2, 'parents!!':1, 'all': 2, 'puppet.': ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2, 'cute': 1,'help': 2, 'doll': 1, ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1, 'be': 1,'is': 1, 'it': 1, 'as': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'feeding,': 1, 'and': 2,'all': 1, 'right': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1, 'help': 1,'give': 1, 'is': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1, 'all': 1,'standarad': 1, ..."


In [5]:
graphlab.canvas.set_target('ipynb')

In [6]:
products['rating'].show()
#print products['rating']

#Examining the reviews for most-sold product:  'Vulli Sophie the Giraffe Teether'

In [194]:
giraffe_reviews = products[products['name'] == 'Vulli Sophie the Giraffe Teether']

In [195]:
len(giraffe_reviews)

785

In [196]:
giraffe_reviews['rating'].show(view='Categorical')

#Build a sentiment classifier

In [7]:
products['rating'].show(view='Categorical')

##Define what's a positive and a negative sentiment

We will ignore all reviews with rating = 3, since they tend to have a neutral sentiment.  Reviews with a rating of 4 or higher will be considered positive, while the ones with rating of 2 or lower will have a negative sentiment.   

In [8]:
#ignore all 3* reviews
products = products[products['rating'] != 3]

In [9]:
#positive sentiment = 4* or 5* reviews
products['sentiment'] = products['rating'] >=4

##Let's train the sentiment classifier

In [10]:
train_data,test_data = products.random_split(.8, seed=0)

In [11]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=['word_count'],
                                                     validation_set=test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 219217
PROGRESS: Number of coefficients    : 219218
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 5        | 0.000002  | 2.717550     | 0.841481          | 0.839989            |
PROGRESS: | 2         | 9        | 3.000000  | 4.360094     | 0.947425          | 0.894877            |
PROGRESS: | 3         | 10       | 3.000000  | 4.974734     | 0.92

#Evaluate the sentiment model

In [12]:
sentiment_model.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +------------------+----------------+-----------------+-------+------+
 |    threshold     |      fpr       |       tpr       |   p   |  n   |
 +------------------+----------------+-----------------+-------+------+
 |       0.0        | 0.224328890558 | 0.0036772581221 | 28010 | 5327 |
 | 0.0010000000475  | 0.775671109442 |  0.996322741878 | 28010 | 5327 |
 | 0.00200000009499 | 0.735686127276 |  0.995287397358 | 28010 | 5327 |
 | 0.00300000002608 | 0.712971653839 |  0.994644769725 | 28010 | 5327 |
 | 0.00400000018999 | 0.69757837432  |  0.994144948233 | 28010 | 5327 |
 | 0.00499999988824 | 0.686314999061 |  0.993787932881 | 28010 | 5327 |
 | 0.00600000005215 | 0.676553407171 |  0.993288111389 | 28010 | 5327 |
 | 0.00700000021607 | 0.666040923597 |  0.992931096037 | 28010 | 5327 |
 | 0.00800000037998 | 0.655716162943 |  0.992681185291 | 28010 | 5327 |
 | 0.00899999961257 | 0.64839496

In [54]:
sentiment_model.show(view='Evaluation')

#Applying the learned model to understand sentiment for Giraffe

In [205]:
giraffe_reviews['predicted_sentiment'] = sentiment_model.predict(giraffe_reviews, output_type='probability')

In [13]:
giraffe_reviews.head()

SyntaxError: unexpected EOF while parsing (<ipython-input-13-5af05b58209d>, line 1)

##Sort the reviews based on the predicted sentiment and explore

In [29]:
giraffe_reviews = giraffe_reviews.sort('predicted_sentiment', ascending=False)

In [14]:
giraffe_reviews.head()

SyntaxError: unexpected EOF while parsing (<ipython-input-14-5af05b58209d>, line 1)

##Most positive reviews for the giraffe

In [22]:
giraffe_reviews[0]['review']

"Sophie, oh Sophie, your time has come. My granddaughter, Violet is 5 months old and starting to teeth. What joy little Sophie brings to Violet. Sophie is made of a very pliable rubber that is sturdy but not tough. It is quite easy for Violet to twist Sophie into unheard of positions to get Sophie into her mouth. The little nose and hooves fit perfectly into small mouths, and the drooling has purpose. The paint on Sophie is food quality.Sophie was born in 1961 in France. The maker had wondered why there was nothing available for babies and made Sophie from the finest rubber, phthalate-free on St Sophie's Day, thus the name was born. Since that time millions of Sophie's populate the world. She is soft and for babies little hands easy to grasp. Violet especially loves the bumpy head and horns of Sophie. Sophie has a long neck that easy to grasp and twist. She has lovely, sizable spots that attract Violet's attention. Sophie has happy little squeaks that bring squeals of delight from Viol

In [23]:
giraffe_reviews[1]['review']

"I'm not sure why Sophie is such a hit with the little ones, but my 7 month old baby girl is one of her adoring fans.  The rubber is softer and more pleasant to handle, and my daughter has enjoyed chewing on her legs and the nubs on her head even before she started teething.  She also loves the squeak that Sophie makes when you squeeze her.  Not sure what it is but if Sophie is amongst a pile of her other toys, my daughter will more often than not reach for Sophie.  And I have the peace of mind of knowing that only edible and safe paints and materials have been used to make Sophie, as opposed to Bright Starts and other baby toys made in China.  Now that the research is out on phthalates and other toxic substances in baby toys, I think it's more important than ever to find good quality toys that are also safe for our babies to handle and put in their mouths.  Sophie is a must-have for every new mom in my opinion.  Even if your kid is one of the few that can take or leave her, it's worth

##Show most negative reviews for giraffe

In [24]:
giraffe_reviews[-1]['review']

"My son (now 2.5) LOVED his Sophie, and I bought one for every baby shower I've gone to. Now, my daughter (6 months) just today nearly choked on it and I will never give it to her again. Had I not been within hearing range it could have been fatal. The strange sound she was making caught my attention and when I went to her and found the front curved leg shoved well down her throat and her face a purply/blue I panicked. I pulled it out and she vomited all over the carpet before screaming her head off. I can't believe how my opinion of this toy has changed from a must-have to a must-not-use. Please don't disregard any of the choking hazard comments, they are not over exaggerated!"

## Building awesome count function

In [15]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']
        

In [46]:
def awesome_count(dicts):
    word = 'and'
    if word in dicts:
        return dicts[word]
    else:
        return 0

# automation function
""""def automate(word_list, products):
    for i in xrange(len(word_list)):
        print word_list[i]
        products[word_list[i]]= products['word_count'].apply(awesome_count)     
 """"  

In [47]:
automate(selected_words, products)

awesome
great
fantastic
amazing
love
horrible
bad
terrible
awful
wow
hate


In [127]:
def awesome_count(dicts):
    word = 'hate'
    if word in dicts:
        return dicts[word]
    else:
        return 0
    

## Buildind the awesome feature column

In [19]:
def awesome_count(dicts):
    word = 'awesome'
    if word in dicts:
        return dicts[word]
    else:
        return 0
products['awesome'] = products['word_count'].apply(awesome_count)


## Add 10 more new features

In [21]:
def awesome_count(dicts):
    word = 'great'
    if word in dicts:
        return dicts[word]
    else:
        return 0
## Great
products['great'] = products['word_count'].apply(awesome_count)


In [22]:
def awesome_count(dicts):
    word = 'fantastic'
    if word in dicts:
        return dicts[word]
    else:
        return 0
# Fantastic
products['fantastic'] = products['word_count'].apply(awesome_count)

In [23]:
def awesome_count(dicts):
    word = 'amazing'
    if word in dicts:
        return dicts[word]
    else:
        return 0
## amazing
products['amazing'] = products['word_count'].apply(awesome_count, skip_undefined=True)


In [24]:
def awesome_count(dicts):
    word = 'love'
    if word in dicts:
        return dicts[word]
    else:
        return 0
## love
products['love'] = products['word_count'].apply(awesome_count)

In [25]:
def awesome_count(dicts):
    word = 'horrible'
    if word in dicts:
        return dicts[word]
    else:
        return 0
## horrible
products['horrible'] = products['word_count'].apply(awesome_count)

In [26]:
def awesome_count(dicts):
    word = 'bad'
    if word in dicts:
        return dicts[word]
    else:
        return 0
## bad
products['bad'] = products['word_count'].apply(awesome_count)

In [27]:
def awesome_count(dicts):
    word = 'terrible'
    if word in dicts:
        return dicts[word]
    else:
        return 0
## terrible
products['terrible'] = products['word_count'].apply(awesome_count)

In [28]:
def awesome_count(dicts):
    word = 'awful'
    if word in dicts:
        return dicts[word]
    else:
        return 0
## awful
products['awful'] = products['word_count'].apply(awesome_count)

In [29]:
def awesome_count(dicts):
    word = 'wow'
    if word in dicts:
        return dicts[word]
    else:
        return 0
## wow
products['wow'] = products['word_count'].apply(awesome_count)

In [30]:
def awesome_count(dicts):
    word = 'hate'
    if word in dicts:
        return dicts[word]
    else:
        return 0
## hate
products['hate'] = products['word_count'].apply(awesome_count)


In [31]:
products

name,review,rating,word_count,sentiment,awesome
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3, 'love': 1,'it': 2, 'highly': 1, ...",1,0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2, 'quilt': 1,'it': 1, 'comfortable': ...",1,0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1, 'and':3, 'love': 2, ...",1,0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2, 'parents!!':1, 'all': 2, 'puppet.': ...",1,0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2, 'cute': 1,'help': 2, 'doll': 1, ...",1,0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1, 'be': 1,'is': 1, 'it': 1, 'as': ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'feeding,': 1, 'and': 2,'all': 1, 'right': 1, ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1, 'help': 1,'give': 1, 'is': 1, ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1, 'all': 1,'standarad': 1, ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 1, 'forget': 1,'just': 1, ""daughter's"": ...",1,0


In [32]:
# Sum of differents words 
result = []
for i in xrange(len(selected_words)):
    print selected_words[i]
    print products[selected_words[i]].sum()

    result.append(products[selected_words[i]].sum())
    
print result


awesome
2002
great
42420
fantastic
873
amazing
1305
love
40277
horrible
659
bad
3197
terrible
673
awful
345
wow
131
hate
1057
[2002, 42420, 873, 1305, 40277, 659, 3197, 673, 345, 131, 1057]


In [33]:
train_data,test_data = products.random_split(.8, seed=0)

## Build a new sentiment analysis model using selected_words

In [34]:
selected_words_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=selected_words,
                                                     validation_set=test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 11
PROGRESS: Number of unpacked features : 11
PROGRESS: Number of coefficients    : 12
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 2        | 0.392447     | 0.844299          | 0.842842            |
PROGRESS: | 2         | 3        | 0.613884     | 0.844186          | 0.842842            |
PROGRESS: | 3         | 4        | 0.855533     | 0.844276          | 0.843142            |
PROGRESS: | 4         | 5        |

In [36]:
coef = selected_words_model['coefficients']
coef.print_rows(12,4)

+-------------+-------+-------+------------------+
|     name    | index | class |      value       |
+-------------+-------+-------+------------------+
| (intercept) |  None |   1   |  1.36728315229   |
|   awesome   |  None |   1   |  1.05800888878   |
|    great    |  None |   1   |  0.883937894898  |
|  fantastic  |  None |   1   |  0.891303090304  |
|   amazing   |  None |   1   |  0.892802422508  |
|     love    |  None |   1   |  1.39989834302   |
|   horrible  |  None |   1   |  -1.99651800559  |
|     bad     |  None |   1   | -0.985827369929  |
|   terrible  |  None |   1   |  -2.09049998487  |
|    awful    |  None |   1   |  -1.76469955631  |
|     wow     |  None |   1   | -0.0541450123333 |
|     hate    |  None |   1   |  -1.40916406276  |
+-------------+-------+-------+------------------+
[12 rows x 4 columns]



In [37]:
coef1 = copy(coef)


NameError: name 'copy' is not defined

In [61]:
selected_words_evaluate = selected_words_model.evaluate(test_data, metric='roc_curve')

In [67]:
selected_words_evaluate.

AttributeError: 'dict' object has no attribute 'Show'